<a href="https://colab.research.google.com/github/chadahun/RAG_LangChain/blob/main/RAG_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install langchain langchain-community faiss-cpu numpy pandas openai wikipedia langchain_openai langchain_core pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 3.5 MB/s eta 0:00:00


In [112]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.runnables import RunnablePassthrough
from langchain.output_parsers import StrOutputParser
import pandas as pd
import numpy as np
import bs4
import os
from google.colab import userdata

ImportError: cannot import name 'RunnablePassthrough' from 'langchain.runnables' (/usr/local/lib/python3.10/dist-packages/langchain/runnables/__init__.py)

In [65]:
os.environ['OPENAI_API_KEY'] = userdata.get('openai_api')

In [26]:
loader = PyPDFLoader('ML.pdf')

In [27]:
docs = loader.load()

In [32]:
print(docs)

[Document(metadata={'source': 'ML.pdf', 'page': 0}, page_content='1 장  한눈에  보는  머신러닝1 장  한눈에  보는  머신러닝'), Document(metadata={'source': 'ML.pdf', 'page': 1}, page_content='감사의  글감사의  글\n자료를  공개한  저자  오렐리앙  제롱에게  깊은  감사를  드립니다 . 이와  더불어  한빛미디어로부터  강의준비에\n필요한  자료를  지원받았음을  밝히며 , 이에  대해  진심어린  감사를  전합니다 .'), Document(metadata={'source': 'ML.pdf', 'page': 2}, page_content='모든  데이터  과학자가  꼭  알아야  할  여러  가지  기초  개념과  용어  설명'), Document(metadata={'source': 'ML.pdf', 'page': 3}, page_content='1.1 머신러닝이란 ?1.1 머신러닝이란 ?'), Document(metadata={'source': 'ML.pdf', 'page': 4}, page_content='머신러닝은  명시적인  프로그래밍 없이  컴퓨터가  학습하는  능력을  갖추게  하는  연구  분야'), Document(metadata={'source': 'ML.pdf', 'page': 5}, page_content='머신러닝  프로그램  예제머신러닝  프로그램  예제\n스팸  필터\n스팸 (spam) 과  스팸이  아닌  메일 (ham) 의  샘플을  이용하여  스팸메일  구분법  학습'), Document(metadata={'source': 'ML.pdf', 'page': 6}, page_content='기본  용어기본  용어\n훈련  세트 (training set): 머신러닝  프로그램이  훈련 ( 학습 ) 하는  데  사용하는  데이터  집합\n훈련  사례  혹은  샘플 (training instance): 각각의  훈련용  데

In [100]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30, is_separator_regex=True)

In [101]:
splits = text_splitter.split_documents(docs)

In [102]:
splits

[Document(metadata={'source': 'ML.pdf', 'page': 0}, page_content='1 장  한눈에  보는  머신러닝1 장  한눈에  보는  머신러닝'),
 Document(metadata={'source': 'ML.pdf', 'page': 1}, page_content='감사의  글감사의  글\n자료를  공개한  저자  오렐리앙  제롱에게  깊은  감사를  드립니다 . 이와  더불어  한빛미디어로부터  강의준비에'),
 Document(metadata={'source': 'ML.pdf', 'page': 1}, page_content='필요한  자료를  지원받았음을  밝히며 , 이에  대해  진심어린  감사를  전합니다 .'),
 Document(metadata={'source': 'ML.pdf', 'page': 2}, page_content='모든  데이터  과학자가  꼭  알아야  할  여러  가지  기초  개념과  용어  설명'),
 Document(metadata={'source': 'ML.pdf', 'page': 3}, page_content='1.1 머신러닝이란 ?1.1 머신러닝이란 ?'),
 Document(metadata={'source': 'ML.pdf', 'page': 4}, page_content='머신러닝은  명시적인  프로그래밍 없이  컴퓨터가  학습하는  능력을  갖추게  하는  연구  분야'),
 Document(metadata={'source': 'ML.pdf', 'page': 5}, page_content='머신러닝  프로그램  예제머신러닝  프로그램  예제\n스팸  필터\n스팸 (spam) 과  스팸이  아닌  메일 (ham) 의  샘플을  이용하여  스팸메일  구분법  학습'),
 Document(metadata={'source': 'ML.pdf', 'page': 6}, page_content='기본  용어기본  용어\n훈련  세트 (training set): 머신러닝  프로그램이  훈련 ( 

In [66]:
vector_store = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [69]:
# vector_store.save_local('VectorStore')

In [ ]:
# FAISS.load_local('VectorStore', OpenAIEmbeddings())

In [103]:
retriever = vector_store.as_retriever(search_kargs={'k': 2})

In [104]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)

In [105]:
template = '''
다음 정보에 기반하여 질문에 답변하세요: {context}

질문: {question}
답변:
'''

In [106]:
prompt = PromptTemplate(input_variables=['context', 'question'], template=template)

In [107]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)

In [108]:
query = '머신러닝의 학습 종류에는 무엇이 있나요?'
res = chain({'query': query})

In [109]:
print('답변:', res['result'])
print('참조: ', res['source_documents'])

답변: 머신러닝의 학습 종류에는 지도 학습, 비지도 학습, 준지도 학습, 강화 학습 등이 있습니다.
참조:  [Document(metadata={'source': 'ML.pdf', 'page': 3}, page_content='1.1 머신러닝이란 ?1.1 머신러닝이란 ?'), Document(metadata={'source': 'ML.pdf', 'page': 17}, page_content='1.4 머신러닝  시스템의  종류1.4 머신러닝  시스템의  종류'), Document(metadata={'source': 'ML.pdf', 'page': 7}, page_content='1.2 왜  머신러닝을  사용하는가 ?1.2 왜  머신러닝을  사용하는가 ?'), Document(metadata={'source': 'ML.pdf', 'page': 4}, page_content='머신러닝은  명시적인  프로그래밍 없이  컴퓨터가  학습하는  능력을  갖추게  하는  연구  분야')]


In [110]:
query = '비지도 학습의 종류를 설명해주세요'
res = chain({'query': query})

In [111]:
print('답변: ', res['result'])
print('참조: ', res['source_documents'])

답변:  비지도 학습의 종류에는 군집, 시각화, 차원 축소, 연관 규칙 학습 등이 있습니다. 군집은 데이터를 비슷한 특성을 가진 그룹으로 나누는 것을 의미하고, 시각화는 데이터를 시각적으로 표현하여 패턴을 찾는 것을 의미합니다. 차원 축소는 데이터의 특성을 줄여서 더 간단하게 만드는 것을 의미하며, 연관 규칙 학습은 데이터 간의 관계를 찾아내는 것을 의미합니다.
참조:  [Document(metadata={'source': 'ML.pdf', 'page': 29}, page_content='비지도  학습비지도  학습\n레이블  없는  훈련  데이터를  이용하여  시스템  스스로  학습'), Document(metadata={'source': 'ML.pdf', 'page': 30}, page_content='대표적  비지도  학습\n군집\n시각화\n차원  축소\n연관  규칙  학습'), Document(metadata={'source': 'ML.pdf', 'page': 23}, page_content='지도  학습지도  학습'), Document(metadata={'source': 'ML.pdf', 'page': 42}, page_content='준지도  학습준지도  학습\n적은  수의  샘플에만  레이블  적용\n비지도  학습을  통해  군집을  분류한  후  샘플들을  활용해  지도  학습을  시킴\n대부분  지도  학습과  비지도  학습  혼합  사용')]


In [126]:
rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [128]:
query = '머신러닝의 장점을 설명해주세요.'

full_response=''

for partial_response in rag_chain.stream(query):
     full_response += partial_response

print(full_response)

머신러닝의 장점은 다음과 같습니다:
1. 전통적인 방식으로는 해결 방법이 없는 복잡한 문제를 해결할 수 있다.
2. 머신러닝 시스템은 새로운 데이터에 쉽게 적응할 수 있다.
3. 복잡한 문제와 대량의 데이터에서 통찰을 얻을 수 있어 데이터 마이닝이 가능하다.
